# Lesson 5b project: Monte Carlo simulations

## Problem statement

**Goal:** Sample random number pairs $x \in (-1, 1)$, $y \in (-1, 1)$ with a uniform distribution to compute the area of a circle with radius $r = 1$. Then use the area $A = \pi r^2$ formula to derive $\pi$.

* How does the accuracy depend on number of sampled pairs?
* What about the computation time?

**Stretch goal:** Use the same technique to compute the area of the Mandelbrot set. The [Mandelbrot set](https://en.wikipedia.org/wiki/Mandelbrot_set) is a set of [complex numbers](https://en.wikipedia.org/wiki/Complex_number) $c$ for which

$$z_{i + 1} = |z_i|^2 + c \mbox{ with } z_0 = 0$$

does not diverge to infinity ($z_i \to \infty$ as $i \to \infty$).

The following CUDA device function identifies whether a point $c = x + y\,i$ is in the Mandelbrot set ($x$ and $y$ are both real numbers and $i = \sqrt{-1}$), using [Floyd's algorithm](https://en.wikipedia.org/wiki/Cycle_detection#Floyd's_tortoise_and_hare) to detect cycles and convergence to zero. This is more accurate than the 20 iteration cut-off we used in [lesson-4-scaling/project-1-mandelbrot.ipynb](../lesson-4-scaling/project-1-mandelbrot.ipynb), which is necessary when computing the area.

```python
@nb.cuda.jit(device=True)
def is_in_mandelbrot(x, y):
    c = np.complex64(x) + np.complex64(y)*np.complex64(1j)
    z_hare = z_tortoise = np.complex64(0)
    while True:
        z_hare = z_hare*z_hare + c
        z_hare = z_hare*z_hare + c
        z_tortoise = z_tortoise*z_tortoise + c
        if z_hare == z_tortoise:
            return True      # orbiting or converging to zero
        if z_hare.real**2 + z_hare.imag**2 > 4:
            return False     # diverging to infinity
```

You can assume that the Mandelbrot set is entirely contained within $x \in (-2, 1)$, $y \in (-\frac{3}{2}, \frac{3}{2})$, which has area $9$ (see [[Knill (2023)](https://doi.org/10.48550/arXiv.2305.17848)], section 4.7).

The exact area of the Mandelbrot set is not known, mathematically. There is an expression,

$$\mbox{area of Mandelbrot set} = \pi \left( 1 - \sum_{n=1}^\infty n \, {b_n}^2 \right)$$

in which the terms $b_n$ can be determined recursively, but it converges very slowly: $10^{118}$ terms are needed to get the first 2 digits, and $10^{1181}$ terms are needed to get 3 digits [[Ewing & Schober (1992)](https://doi.org/10.1007/BF01385497)]. The best estimates of the Mandelbrot set's area come from sampling techniques like this one. The most recent publication is [[Bittner, Cheong, Gates, & Nguyen (2012)](https://doi.org/10.2140/involve.2017.10.555)] and the most recent unpublished estimate is [[Förstemann (2017)](https://www.foerstemann.name/labor.html)] using 2× Radeon HD 5970 and a tree-splitting (rather than random) search. The most precise, rigorous bounds to date are

$$1.50640 < \mbox{area of Mandelbrot set} < 1.53121\mbox{.}$$

If you're interested in this sort of thing, Robert Munafo wrote a [rabbit warren of hyperlinked pages](http://www.mrob.com/pub/muency/areaofthemandelbrotset.html) about all of the techniques in 2003, from a [Usenet thread (alt.fractals)](https://ics.uci.edu/~eppstein/junkyard/mand-area.html) that started exactly 5 days after the [first release of Python (alt.sources)](https://www.tuhs.org/Usenet/alt.sources/1991-February/001749.html). Weird, huh?